In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.4.11 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(16, 3)

In [4]:
dataset = pd.DataFrame(
    {
        "Euler1": database.euler[:, 0],
        "Euler2": database.euler[:, 1],
        "Euler3": database.euler[:, 2],
    }
)
dataset

,Euler1,Euler2,Euler3
0,0,0,0
1,0,0,45
2,0,0,90
3,0,0,135
4,0,45,0
5,0,45,45
6,0,45,90
7,0,45,135
8,45,0,0
9,45,0,45


In [5]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [6]:
# create a random generator
rgn = np.random.default_rng()

# set standard deviations for T and azimuths
std_T = 0.03
std_azimuths = 0.5

In [7]:
# just a test
np.arange(0, 180, 9.0)

array([  0.,   9.,  18.,  27.,  36.,  45.,  54.,  63.,  72.,  81.,  90.,
        99., 108., 117., 126., 135., 144., 153., 162., 171.])

In [8]:
steps = (13.0, 11.5, 9.0, 7.5)  # in degrees
columns = ["n14", "n16", "n20", "n24"]

for i, step in enumerate(steps):
    # create database
    angles = np.arange(0, 180, step)
    T_vals = []
    azi_vals = []

    for euler in database.euler:
        # rotate
        x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

        # extract XY intersection
        xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

        # get the indexes of specific angles
        indexes = f.find_nearest(xy_vectors["angles"], angles)

        # append values
        T_vals.append(xy_vectors.loc[indexes, ["T"]].T.values.tolist()[0])
        azi_vals.append(xy_vectors.loc[indexes, ["angles"]].T.values.tolist()[0])

    database.T_values = np.array(T_vals)
    database.azimuths = np.array(azi_vals)

    # add Gaussian noise to data
    database.T_noise = database.T_values + rgn.normal(0, std_T, database.T_values.shape)
    database.azi_noise = database.azimuths + rgn.normal(
        0, std_azimuths, database.azimuths.shape
    )

    # initialize variables
    no_noise = np.empty(database.euler.shape[0])
    noise = np.empty(database.euler.shape[0])

    for index, orientation in enumerate(database.euler):
        measures1 = np.column_stack(
            (
                database.T_values[index],
                database.azimuths[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate1 = f.find_orientation(measurements=measures1, params=(Ta, Tb, Tc))
        no_noise[index] = f.calc_disorientation(orientation, estimate1.x)

        measures2 = np.column_stack(
            (
                database.T_noise[index],
                database.azi_noise[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate2 = f.find_orientation(measurements=measures2, params=(Ta, Tb, Tc))
        noise[index] = f.calc_disorientation(orientation, estimate2.x)

    # add columns
    dataset[columns[i]] = no_noise
    dataset[columns[i] + "_noise"] = noise

    # clean database
    del database.T_values
    del database.azimuths
    del database.T_noise
    del database.azi_noise


In [9]:
# check
# print('Euler angles: ', database.euler[-1])
# print('T values: ', np.around(database.T_values[-1], 2))
# print('Azimuths: ', np.around(database.azimuths[-1], 1))
# print('T noise: ', np.around(database.T_noise[-1], 2))
# print('Azi_noise: ', np.around(database.azi_noise[-1], 1))

In [10]:
dataset

,Euler1,Euler2,Euler3,n14,n14_noise,n16,n16_noise,n20,n20_noise,n24,n24_noise
0,0,0,0,0.000,13.316,0.000,10.367,0.289,9.285,0.262,9.348
1,0,0,45,0.000,0.841,0.000,5.980,0.000,18.034,0.000,6.460
2,0,0,90,0.000,13.467,0.000,8.062,0.303,0.504,0.268,11.779
3,0,0,135,0.000,8.202,0.288,7.375,0.000,15.772,0.275,0.509
4,0,45,0,0.003,11.754,0.000,5.472,0.000,2.237,0.001,9.085
5,0,45,45,0.001,2.010,0.070,5.108,0.007,8.639,0.001,4.036
6,0,45,90,0.000,7.354,0.029,2.598,0.001,12.296,0.000,3.142
7,0,45,135,0.002,13.060,0.005,1.928,0.006,2.687,0.005,5.001
8,45,0,0,0.000,7.489,0.000,9.029,0.361,20.059,0.282,12.833
9,45,0,45,0.389,9.758,0.000,11.811,0.230,12.495,0.289,0.134


## Estimate

In [11]:
from datetime import date    
today = date.today().isoformat()

In [12]:
dataset.to_csv(today + '_step7_lamb1987_noise_datapoints.csv', index=False)

In [13]:
# quick statistical overview
dataset[['n14', 'n14_noise', 'n16', 'n16_noise', 'n20', 'n20_noise', 'n24', 'n24_noise']].describe()

,n14,n14_noise,n16,n16_noise,n20,n20_noise,n24,n24_noise
count,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.037813,7.92550,0.025438,6.571312,0.094250,7.454062,0.087312,4.471063
std,0.104891,5.01442,0.072302,4.504435,0.142267,6.617334,0.130956,4.224158
min,0.000000,0.84100,0.000000,1.868000,0.000000,0.504000,0.000000,0.134000
25%,0.000000,2.67075,0.000000,2.796750,0.000750,1.988000,0.000750,0.645500
50%,0.001500,7.89100,0.001500,5.726000,0.004500,5.880500,0.003000,3.070500
75%,0.005000,12.08050,0.004250,8.303750,0.244750,12.345750,0.263500,7.116250
max,0.389000,16.92200,0.288000,19.045000,0.361000,20.059000,0.289000,12.833000
